This notebook goes with [this blog post](https://sgugger.github.io/pointer-cache-for-language-model.html#pointer-cache-for-language-model) that explains what the continuous cache pointer is. This technique was introduce by Grave et al. in [this article](https://arxiv.org/pdf/1612.04426.pdf).

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import dill as pickle
import json
from IPython.display import Image
from IPython.core.display import HTML
from hazm import *
from fastai.text import *

from glob import glob
import re
from pathlib import Path


In [2]:
PATH = 'extract/AA/'

## Language Model

For more information on how I created the train and validation indexes look at this [jupyter notebook](https://github.com/layla-tadjpour/Deep-Learning/blob/master/language_model_persian.ipynb).

In [3]:
LM_PATH=Path('extract/AA/persian_lm/')
LM_PATH.mkdir(exist_ok=True)

In [4]:
trn_lm = np.load(LM_PATH/'tmp'/'trn_ids_hazm.npy')
val_lm = np.load(LM_PATH/'tmp'/'val_ids_hazm.npy')
itos = pickle.load(open(LM_PATH/'tmp'/'itos_hazm.pkl', 'rb'))

In [5]:
vs=len(itos)
vs,len(trn_lm)

(60002, 188257)

In [6]:
stoi = collections.defaultdict(lambda: 0, {v:k for k,v in enumerate(itos)})
len(itos),stoi['سلام']

(60002, 5046)

In [7]:
em_sz,nh,nl = 400,1150,3
wd=1e-7
bptt=70
bs=52
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))

In [8]:
trn_dl = LanguageModelLoader(np.concatenate(trn_lm), bs, bptt)
val_dl = LanguageModelLoader(np.concatenate(val_lm), bs, bptt)
md = LanguageModelData(PATH, 1, vs, trn_dl, val_dl, bs=bs, bptt=bptt)

In [9]:
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7

In [10]:
learner= md.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

The model I use as en example is stored here. Be sure to have the file best.h5 in a directory called models where the variable PATH points to (our replace by any model you've saved).

In [11]:
learner.load('lm_hazm_ft_after_32epochs')

Let's begin by computing how well our model is doing before anything else. To do that we will need a way to go through all of our text, but instead of using the fastai LanguageModelLoader (who randomly modifies the bptt) we'll change the code to have a fixed bptt.

Also we don't want to do mini-batches on this validation because it resets the hidden state at each batch, making us lose valuable information. It makes a tiny bit of difference as we will see.

In [12]:
#Comes from the LanguageModelLoader class, I just removed the minibatch and fixed the bptt.
#Now it gives an iterator that will spit bits of size bptt.
class TextReader():
    def __init__(self, nums, bptt, backwards=False):
        self.bptt,self.backwards = bptt,backwards
        self.data = self.batchify(nums)
        self.i,self.iter = 0,0
        self.n = len(self.data)

    def __iter__(self):
        self.i,self.iter = 0,0
        while self.i < self.n-1 and self.iter<len(self):
            res = self.get_batch(self.i, self.bptt)
            self.i += self.bptt
            self.iter += 1
            yield res

    def __len__(self): return self.n // self.bptt 

    def batchify(self, data):
        data = np.array(data)[:,None]
        if self.backwards: data=data[::-1]
        return T(data)

    def get_batch(self, i, seq_len):
        source = self.data
        seq_len = min(seq_len, len(source) - 1 - i)
        return source[i:i+seq_len], source[i+1:i+1+seq_len].view(-1)

This TextReader will give us an iterator that will allow us to go through the text. 

In [13]:
def my_validate(model, source, bptt=2000):
    data_source = TextReader(source, bptt)
    model.eval()
    model.reset()
    total_loss = 0.
    for inputs, targets in tqdm(data_source):
        #The language model throws up a bucnh of things, we'll focus on that later. For now we just want the ouputs.
        outputs, raws, outs = model(V(inputs))
        #The output doesn't go through softmax so we can use the CrossEntropy loss directly 
        total_loss += F.cross_entropy(outputs, V(targets), size_average=False).data[0]
    #Total size is length of our iterator times bptt
    mean = total_loss / (bptt * len(data_source))
    #Returns loss and perplexity.
    return mean, np.exp(mean)

In [20]:
val_lm.shape

(49052,)

In [30]:
my_validate(learner.model, np.concatenate(val_lm))

100%|██████████| 204595/204595 [49:07<00:00, 69.41it/s]


(4.043599833351682, 57.03127681238258)

In [22]:
def one_hot(vec, size=vs, cuda=True):
    a = torch.zeros(len(vec), size)
    for i,v in enumerate(vec):
        a[i,v] = 1.
    return V(a)

In [33]:
def my_cache_pointer(model, source, theta = 0.662, lambd = 0.1279, window=200, bptt=2000):
    data_source = TextReader(source, bptt)
    #Set the model into eval mode.
    model.eval()
    #Just to create a hidden state.
    model.reset()
    total_loss = 0.
    #Containers for the previous targets/hidden states.
    targ_history = None
    hid_history = None
    for inputs, targets in tqdm(data_source):
        outputs, raws, outs = model(V(inputs))
        #The outputs aren't softmaxed, sowe have to do it to get the p_vocab vectors.
        p_vocab = F.softmax(outputs)
        #We take the last hidden states (raws contains one Tensor for the results of each layer) and remove the batch dimension.
        hiddens = raws[-1].squeeze() 
        #Start index inside our history.
        start = 0 if targ_history is None else targ_history.size(0)
        #Add the targets and hidden states to our history.
        targ_history = one_hot(targets) if targ_history is None else torch.cat([targ_history, one_hot(targets)])
        hid_history = hiddens if hid_history is None else torch.cat([hid_history, hiddens])
        for i, pv in enumerate(p_vocab):
            #Get the cached values
            p = pv
            if start + i > 0:
                targ_cache = targ_history[:start+i] if start + i <= window else targ_history[start+i-window:start+i]
                hid_cache = hid_history[:start+i] if start + i <= window else hid_history[start+i-window:start+i]
                #This is explained in the blog post.
                all_dot_prods = torch.mv(theta * hid_cache, hiddens[i])
                softmaxed = F.softmax(all_dot_prods).unsqueeze(1)
                p_cache = (softmaxed.expand_as(targ_cache) * targ_cache).sum(0).squeeze()
                p = (1-lambd) * pv + lambd * p_cache
            total_loss -= torch.log(p[targets[i]]).data[0]
        targ_history = targ_history[-window:]
        hid_history = hid_history[-window:]
    #Total size is length of our iterator times bptt
    mean = total_loss / (bptt * len(data_source))
    #Returns loss and perplexity
    return mean, np.exp(mean)

In [34]:
my_cache_pointer(learner.model, np.concatenate(val_lm))

100%|██████████| 7160/7160 [3:27:16<00:00,  1.74s/it]  


(3.9760679426004084, 53.30701535078493)

So, we went from 57.03 perplexity to 53.30. This result can be imporved by increasing the window length, though it would take longer to evaluate the model.

## Test

Let's test the model to predict next words in farsi for texts of various lenghts.

In [35]:
m = learner.model

In [36]:
def proc_str(s): return Tokenizer().spacy_tok(s)

In [37]:
def num_str(s): 
    idx_arr = np.array([stoi[tok] for tok in proc_str(s)])
    return torch.from_numpy(np.expand_dims(idx_arr,axis=1)).cuda()

In [38]:
def sample_model(m, s, l=50):
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(Variable(t))
    print('...', end='')

    for i in range(l):
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        word = itos[n.data[0]]
        print(word, end=' ')
        #if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0))

    m[0].bs=bs

In [39]:
ss = """

به نظرم توجه به رضا شاه نوعی نومیدی از شرایطی است
که نهاد دولت را از اقتدار تهی کرده است به طوری که توان
تصمیم گیری برای حل معضلات کشور و یا اجرای تصمیمات خود را 
ندارد و کارش به "حرف درمانی" تقلیل یافته است. 
در واقع با کمی تأمل می توان دریافت که کشور نه فقط دستخوش نوعی
از ملوک الطوایفی است و مسئولان هر استانی ساز خود را می نوازند،
بلکه در هر شهر و آبادی نیز دهها نهاد و دستگاه با تعریف منافع
و رانت های مشخص اقتصادی و سیاسی و مدیریتی برای خود و اطرافیان شان، در جهت خنثی ساز فعالیت های یکدیگر در نزاع و رقابت اند و از این جهت نه فقط مشکلی را حل نمی کنند بلکه به انباشت روزافزون مشکلات دامن می زنند.


"""

In [40]:
sample_model(m,ss,l=50)

...در این زمینه می گوید که در این دوره از تاریخ ایران ، ایران به عنوان یک کشور مستقل و مستقل از ایران و جهان شناخته می شود و در این زمینه به عنوان یک کشور مستقل و مستقل در نظر گرفته_می‌شود که در آن کشور به عنوان یک کشور 

In [41]:
ss= """
پیش از این هم نوشته بودم که در شرایط انباشت مشکلات،

یک فرد عادی جامعه در پی دمکراسی و این قبیل سخنان نیست؛ او صرفاً می خواهد که مشکلاتش از هر طریقی حل شود. در چنین شرایطی، زمینه برای ظهور فرد مقتدر و با صلابت و حتی مستبدی فراهم می شود که کارآیی خود را در عمل نشان دهد و در نزاع بین کانون های متعدد قدرت، تصمیمی فیصله بخش بگیرد. چنین فردی اگر در حوزۀ زندگی خصوصی و فرهنگی و شیوۀ زیست افراد دخالت نکند و آنها را از این جهت به تنگ نیاورد و با دنیا هم به گونه ای راه سازش و مسالمت در پیش گیرد؛ می تواند از اقبال عمومی برخوردار شود.
"""

In [42]:
sample_model(m,ss,l=50)

...، ص در این میان ، در این میان ، در این میان ، در این میان ، در این میان ، در این میان ، به نظر می‌رسد که این دو ، در واقع ، به نوعی از این دو ، در کنار یکدیگر ، به هم پیوند خورده‌اند 

In [43]:
ss= "وای این فخرآور خیلی بانمکه که برای خودش رفته پول داده امضای دونالد ترامپ خریده و بعد به خودش از قول ترامپ گفته: توماس جفرسون ایران. می‌خوام برم یکی عینش رو سفارش بدم بگم دونالد برام بنویسه تو شهناز تهرانی آمریکایی."

In [44]:
sample_model(m,ss,l=20)

...، زاده اکتبر درگذشته مارس کارآفرین ، بازرگان و مدیر ارشد اجرایی بریتانیایی بود ، که در سال شرکت خودروسازی 